In [1]:
import torch
import torchio as tio
import pickle
from mgmt.data.pickle import subject_remove_paths
import os

In [2]:
subject_pkl = "/home/ubuntu/storage/radiology-research-west-2/expr/brain_tumor/preprocess-subjects-v2/resample-2.0-crop-64/P-00003.pkl"

def load_subject(fpath):
    with open(fpath, "rb") as f:
        subject = pickle.load(f)
    subject = subject_remove_paths(subject)
    return subject

In [3]:
subject = load_subject(subject_pkl)

In [14]:
cache_dir = "/home/ubuntu/storage/cache_dir"

class CacheableSubject(tio.Subject):
    def __init__(self, *args, **kwargs):
        cache_dir = kwargs.pop("cache_dir") 
        super().__init__(**kwargs)
        self.cache_path = os.path.join(cache_dir, self.patient_id_str)
        self.cached = False
    
    @classmethod
    def from_subject(cls, subject: tio.Subject, cache_dir: str = ""):
        return CacheableSubject(cache_dir=cache_dir, **dict(subject.items()))
    
    def cache(self):
        images = self.get_images_dict(intensity_only=False)
        with open(self.cache_path, "wb") as f:
            pickle.dump(images, f)
        for name in images.keys():
            self.remove_image(name)
        self.cached = True
    
    def load_cache(self):
        with open(self.cache_path, "rb") as f:
            images = pickle.load(f)
        for name, image in images.items():
            self.add_image(image, name)
        self.cached = False
    
    def load(self):
        if self.cached:
            self.load_cache()
        else:
            super().load()

In [15]:
cache_subject = CacheableSubject.from_subject(subject)

In [16]:
dataset = tio.SubjectsDataset([cache_subject])

In [17]:
dataset[0]

> /tmp/ipykernel_3766/1688838116.py(35)load()
     33             import ipdb
     34             ipdb.set_trace()
---> 35             super().load()

ipdb> s
--Call--
> /home/ubuntu/miniconda3/envs/mgmt/lib/python3.10/site-packages/torchio/data/subject.py(361)load()
    360 
--> 361     def load(self) -> None:
    362         """Load images in subject on RAM."""

ipdb> n
> /home/ubuntu/miniconda3/envs/mgmt/lib/python3.10/site-packages/torchio/data/subject.py(363)load()
    362         """Load images in subject on RAM."""
--> 363         for image in self.get_images(intensity_only=False):
    364             image.load()

ipdb> n
> /home/ubuntu/miniconda3/envs/mgmt/lib/python3.10/site-packages/torchio/data/subject.py(364)load()
    363         for image in self.get_images(intensity_only=False):
--> 364             image.load()
    365 

ipdb> image
LabelMap(shape: (1, 70, 89, 69); spacing: (2.00, 2.00, 2.00); orientation: RAS+; dtype: torch.ByteTensor; memory: 419.8 KiB)
ipdb> n
> /hom

In [ ]:
cache_subject.cache()

In [ ]:
cache_subject

In [ ]:
cache_subject.load_cache()

In [ ]:
cache_subject

In [18]:
48*10

480

In [19]:
40*5

200